In [1]:
# import library
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pulp
import random
import math
import pdb
import time
import os
from shutil import copyfile

In [2]:
os.getcwd()

'D:\\alex\\AMS Project\\SP routing\\Full network - oc6 new'

In [3]:
def read_network():
    '''
        input: network.dat
        output:
            num_zones: number of zones within the network
            num_nodes: number of nodes in the network
            num_links: number of links in the network
            node_detail: node id and zone which the node belong to
            node_id: match the node id to 1~num_nodes tonet save memory
            link_detail:The information associated with a specific link
                link_detail[0] upstream node
                link_detail[1] downstream node
                link_detail[2] Num of left turn bay
                link_detail[3] Num of right turn bay
                link_detail[4] length of link
                link_detail[5] Num of lanes
                link_detail[6] Traffic flow mode
                link_detail[7] posted speed limit adjustment margin
                link_detail[8] posted speed limit
                link_detail[9] maximum service flow rate for the first link
            link_id: a matrix to find a specific link based on origin node nd destination node 
   '''
    global num_zones,num_nodes,num_links,node_detail,node_id,link_detail,link_id,adjacent_link
    file=open('network.dat','r')
    i=0
    network_basic=list()
    node_list=list()
    node_id={}
    link_detail_list=list()
    for line in file:
        line_list_temp=line.split()
        # Read the first line of network.dat
        if i==0:
            num_zones=int(line_list_temp[0])
            print(num_zones)
            num_nodes=int(line_list_temp[1])
            num_links=int(line_list_temp[2]) #+5 # +5 because 5 nodes can not be visited from anyother link, therefore we manually creat few link to connect it to the network
            num_shortest_path=int(line_list_temp[3])
            zone_flag=int(line_list_temp[4])
            link_id=np.zeros((num_nodes,num_nodes))
        #Read the node information from network.dat
        elif len(line_list_temp)<3:
            node_list.append([int(line_list_temp[0]),int(line_list_temp[1])])
            node_id.update({int(line_list_temp[0]):i-1})
        #Read the link information from network.dat
        else:
            line_list_temp=[float(j) for j in line_list_temp]
            link_detail_list.append(line_list_temp)
            link_id[node_id[line_list_temp[0]],node_id[line_list_temp[1]]]=i-num_nodes-1
        i=i+1
    node_detail=np.matrix(node_list)
    link_detail=np.matrix(link_detail_list)
    adjacent_link=[]
    for (link1,i) in zip(link_detail_list,range(len(link_detail_list))):
        
            
        temp=[]
        #find the link that is adjacent to the current link
        for (link2,j) in zip(link_detail_list,range(len(link_detail_list))):
            
            if(link2[0]==link1[1] or link2[1]==link1[0]):
                temp.extend([j])
        adjacent_link.append(temp)
    return  num_zones,num_nodes,num_links,node_detail,node_id,link_detail,link_id,adjacent_link
def read_flow():
    '''
        input: outflow.dat
        output: 
            link_volume - matrix
    '''
    global link_volume
    volume_time=[]
    file=open('OutFlow.dat')
    i=1
    link_volume=[]
    link_volume_one_time_interval=[]
    for line in file:
        line_list=line.split()
        if i>6 : # skip the first 6 lines
            if len(line_list)==1: #find the line indicating time interval
                volume_time.append(float(line_list[0]))
#                 link_volume_one_time_interval.extend([0.01,0.01,0.01,0.01,0.01])  #serveral nodes can not be visited from anyother link, therefore we manually creat few link to connect it to the network
                #print(line_list[0],len(link_volume_one_time_interval))
                link_volume.append(link_volume_one_time_interval)
                link_volume_one_time_interval=[]
            else:
                line_list=[float(j) for j in line_list]
                link_volume_one_time_interval.extend(line_list)
        i=i+1
#     link_volume_one_time_interval.extend([35,35,35,35,35])  #serveral nodes can not be visited from anyother link, therefore we manually creat few link to connect it to the network
    link_volume.append(link_volume_one_time_interval)
    link_volume.pop(0)
    link_volume=np.array(link_volume)
    return link_volume
def read_speed():
    '''
        input:OutLinkSpeedAll.dat
        output: 
            link_speed -matrix
    '''
    global link_speed
    speed_time=[]
    file=open('OutLinkSpeedAll.dat')
    i=1
    link_speed=[]
    link_speed_one_time_interval=[]
    for line in file:
        line_list=line.split()
        if i>6 : # skip the first 6 lines
            if len(line_list)==1: #find the line indicating time interval
                speed_time.append(float(line_list[0]))
#                 link_speed_one_time_interval.extend([0,0,0,0,0])  #serveral nodes can not be visited from anyother link, therefore we manually creat few link to connect it to the network
                link_speed.append(link_speed_one_time_interval)
                link_speed_one_time_interval=[]
            else:
                line_list=[float(j) for j in line_list]
                link_speed_one_time_interval.extend(line_list)
        i=i+1
#     link_speed_one_time_interval.extend([0,0,0,0,0])  #serveral nodes can not be visited from anyother link, therefore we manually creat few link to connect it to the network
    link_speed.append(link_speed_one_time_interval)
    link_speed.pop(0)
    link_speed=np.array(link_speed)
    return link_speed

def read_xy():
    '''
        input: xy.dat
        output: nodexy: a dictionary storing the latitude and longtitude of 
            each node
    '''
    global nodexy
    nodexy={}
    file=open('xy.dat')
    for line in file:
        line_list_temp=line.split()
        line_list_temp=[float(j) for j in line_list_temp]
        nodexy.update({line_list_temp[0]:[line_list_temp[1]/1000000.0*51.33,-line_list_temp[2]/1000000.0*68]})
    return
def read_snow():
    '''
        ipnut: weather.dat
        output: snow_detail. 
                    snow_detail[i,0] snow intensity at time interval i
                    snow_detail[i,1] start time of time interval i
                    snow_detail[i,2] end time of time interval i
    '''
    global snow_detail,snow_interval
    file=open('weather.dat')
    snow_detail=[]
    i=0
    for line in file: 
        if i==0:
            num_interval=float(line.split()[0])
            #print(num_interval,i,line)
            snow_interval=planning_horizon/num_interval
        elif i!= num_interval+1:
            line_list_temp= line.split()
            snow_detail.append([float(line_list_temp[2]),float(line_list_temp[3]),float(line_list_temp[4])])
        i=i+1
    return
def read_snowaccum(): 
    global sa_factors
    sa_factors=[]
    file=open('SnowAccuFactor.dat')
    for line in file: 
        if len(line)>4:
            line_list_temp=[j for j in line.split()]
            sa_factors.append([float(j) for j in line_list_temp])
    return
# def read_scenario()
#     '''
#         Input: Scenario.dat
#         output:
#     '''

In [5]:
def link_class_partrition():
    '''
        input: 
            link_volume
        output:
            link_class
    '''
    global link_class
    #Calculate the threshold of different service level
    service_threshold=[]
    service_threshold.append(np.percentile(np.amax(link_volume,axis=0),80))
    service_threshold.append(np.percentile(np.amax(link_volume,axis=0),40))
    service_threshold.append(np.percentile(np.amax(link_volume,axis=0),0))
    i=0
    link_class=[]
    for link in link_detail:
        if max(link_volume[:,i])>=service_threshold[0] or (link[0,11]!=5) :
            link_class.append(1)
        elif max(link_volume[:,i])>=service_threshold[1]: #or if it is a bus routes, need to modify later
            link_class.append(2)
        else: 
            link_class.append(3)
        i=i+1
    return
            

In [6]:
def distance_between_link(link_ID1,link_ID2,link_xy):
#     xyoflink1=[]
#     xyoflink1.extend([(nodexy[link_detail[link_ID1,0]][0]+nodexy[link_detail[link_ID1,1]][0])/2])
#     xyoflink1.extend([(nodexy[link_detail[link_ID1,0]][1]+nodexy[link_detail[link_ID1,1]][1])/2])
#     xyoflink2=[]
#     xyoflink2.extend([(nodexy[link_detail[link_ID2,0]][0]+nodexy[link_detail[link_ID2,1]][0])/2])
#     xyoflink2.extend([(nodexy[link_detail[link_ID2,0]][1]+nodexy[link_detail[link_ID2,1]][1])/2])
#     distance=math.sqrt((xyoflink1[0]-xyoflink2[0])**2+(xyoflink1[0]-xyoflink2[0])**2)
    distance=math.sqrt((link_xy[link_ID1][0]-link_xy[link_ID2][0])**2+(link_xy[link_ID1][1]-link_xy[link_ID2][1])**2)
    return distance
def c_link_xy (links):
    global link_xy
    link_xy=[]
    for link_ID1 in links: 
        link_xy.append([(nodexy[link_detail[link_ID1,0]][0]+nodexy[link_detail[link_ID1,1]][0])/2,
                       (nodexy[link_detail[link_ID1,0]][1]+nodexy[link_detail[link_ID1,1]][1])/2])
    return 

In [7]:
def net_work_partrition(network_link_list,num_cluster,center,seed_links):
    '''
    input: 
        network_link_list: a list of link id that needs to be partitioned 
        num_cluster: the network would be partitioned into num_cluster sets
        center: optional input, the location of the depot
        seed_links: optional input, the seed link for each cluster
    output
        clusters: a list of link contained in each cluster
    '''
    #global d
    if not center: 
        center=identify_center_depot(network_link_list)
    if not seed_links: # in this case, we do not have a seedlink,but have a center depot
        for i in range(num_cluster):
            maxdistance=0
            linkcandidate=0
            #Go over all the candidate links to find the one which locates furtherest to the depot and other seedlinks
            for link in network_link_list:
                #calculate distance form link candidate to the depot
                distance_to_depot=math.sqrt((nodexy[link_detail[link,0]][0]-nodexy[center][0])**2
                                            +(nodexy[link_detail[link,0]][1]-nodexy[center][1])**2)
                total_distance=distance_to_depot
                #calculate distance form link candidate to other seed links
                for seed_link in seed_links: 
                    total_distance=total_distance*distance_between_link(seed_link,link,link_xy)
                if total_distance>maxdistance and link not in seed_links:
                    maxdistance=total_distance
                    linkcandidate=link
            seed_links.append(linkcandidate)
    print('The following links are selected as seed link: \n',seed_links)
    #######################################################
    if cluster_method==0:
        #cluster based on integer programming
        #After the seed_links are selected, solve the integer problem to identify the link clusters
        cluster_link_index=[i for i in range(len(network_link_list))]
        cluster_zone_index=[i for i in range(num_cluster)]
        link_length=[link_detail[i,4]*link_detail[i,5] for i in network_link_list]
        #link_length=[link_detail[i,4]*link_detail[i,5]/5280 for i in network_link_list]

        L=float(sum(link_length)/num_cluster*0.8)
        U=float(sum(link_length)/num_cluster*1.12)

        prob=pulp.LpProblem('Link Cluster Problem',pulp.LpMinimize)
        link_cluster=pulp.LpVariable.dicts("Cluster",(cluster_link_index,cluster_zone_index),0,1,pulp.LpInteger)
        d=np.zeros(shape=(len(network_link_list),num_cluster))
        for l in cluster_link_index:
            for z in cluster_zone_index:
                #print(l,z,network_link_list[l],seed_links[z])
                d[l,z]=distance_between_link(network_link_list[l],seed_links[z],link_xy)
        prob += pulp.lpSum([d[l,z]*link_cluster[l][z]] for l in cluster_link_index for z in cluster_zone_index), "Total distance from link to assigned seed link"
        for l in cluster_link_index:
            prob += pulp.lpSum([link_cluster[l][z] for z in cluster_zone_index]) == 1
        for z in cluster_zone_index:
            prob += pulp.lpSum([float(link_length[l])*link_cluster[l][z]] for l in cluster_link_index )>=L
            prob += pulp.lpSum([float(link_length[l])*link_cluster[l][z]] for l in cluster_link_index)<=U
        prob.writeLP("Link Cluster Problem.lp")
        # The problem is solved using PuLP's choice of Solver
        prob.solve()
        print("Solution Status:", pulp.LpStatus[prob.status])
        network_link_cluster_list=[[] for i in range(num_cluster)]
        for l in cluster_link_index:
            for z in cluster_zone_index:
                if pulp.value(link_cluster[l][z])==1:
                    network_link_cluster_list[z].append(network_link_list[l])
    
#   end of integer programming clustering
    #############################################################
    
    ##############################################################
    elif cluster_method==1:
    # cluster based on heuristic
        unserved_link=[i for i in network_link_list]
    # initial a set of link should be served by each cluster,network_link_cluster_list. At the beginning it should include only the seed nodes 
        network_link_cluster_list=[[] for i in range(num_cluster)]
        total_cluster_length=[0 for i in range(num_cluster)]
        for seed in seed_links:
            unserved_link.remove(seed)
    # initial a set of link that adjacent to the service cluster, adjacent_link
        network_link_cluster_adj_list=[[] for i in range(num_cluster)]
        for i in range(num_cluster):
            for link in adjacent_link[seed_links[i]]:
                if link in unserved_link:
                    network_link_cluster_adj_list[i].extend([link])


        #upper and lower bound of each cluster
        link_length=[link_detail[i,4]*link_detail[i,5]/5280 for i in network_link_list]
        L=float(sum(link_length)/num_cluster*0.8)
        U=float(sum(link_length)/num_cluster*1.2)
    # create a ranked list of cluster, ranked by the length of total link
    # as long as the service cluster does not cover the whole network continue
        while unserved_link != []:
            #find the longest link adjacent to the smallest cluster
            max_length=0
            min_length=10**30
            min_total_length=10**30
            max_total_length=0
            for i in range(num_cluster):
                if total_cluster_length[i]<min_total_length and network_link_cluster_adj_list[i] !=[]:
    #            if total_cluster_length[i]>max_total_length and network_link_cluster_adj_list[i] !=[]:
                    target_cluster=i
                    min_total_length=total_cluster_length[i]
    #         flag=0
    #         if target_cluster != total_cluster_length.index(min(total_cluster_length)) and flag==0:
    #             pdb.set_trace()
    #             flag=1
            for link in network_link_cluster_adj_list[target_cluster]:
    #             if link_detail[link,4]*link_detail[link,5]/5280 > max_length:
    #                 max_length=link_detail[link,4]*link_detail[link,5]/5280
                if link_detail[link,4]*link_detail[link,5]/5280 < min_length:
                    min_length=link_detail[link,4]*link_detail[link,5]/5280
                    link_candidate=link
            #print('linkcandidate',min_total_length,link_candidate,target_cluster,total_cluster_length.index(min(total_cluster_length)))


            #Add new adjacent links
            network_link_cluster_list[target_cluster].extend([link_candidate])
            network_link_cluster_adj_list[target_cluster].extend(adjacent_link[link_candidate])
            network_link_cluster_adj_list[target_cluster]=list(set(network_link_cluster_adj_list[target_cluster]))

            #Remove the link from unserved link and the all adjacent list
            unserved_link.remove(link_candidate)
            for cluster in network_link_cluster_adj_list:
                if link_candidate in cluster: 
                    cluster.remove(link_candidate)


            for link in adjacent_link[link_candidate]:
                if link not in unserved_link:
                    network_link_cluster_adj_list[target_cluster].remove(link)


            #Update cluster length
            total_cluster_length[target_cluster]=total_cluster_length[target_cluster]+min_length
    
    ##############################################################
    
    return network_link_cluster_list



In [8]:
def identify_center_depot(link_list):
    '''
        input: 
            link_list list of link within the zone
        output; 
            depot_node: a node number where the depot is located
        This subroutine identify a depot which is located at the center of the target zone
    '''
    min_distance=math.inf
    axy=[0,0]
    for link in link_list:
        axy=[axy[0]+link_xy[link][0],axy[1]+link_xy[link][1]]
    axy=[axy[0]/len(link_list),axy[1]/len(link_list)]
    for link in link_list:
        distance=math.sqrt((link_xy[link][0]-axy[0])**2+(link_xy[link][0]-axy[0])**2)
        if distance< min_distance:
            min_distance=distance
            depot_candidat=link
    depot_node=link_detail[depot_candidat,0]
    return depot_node

In [9]:
def cluster_highlight(linklist,title):
    #This function will highlight the linklist in a given network
    plt.figure() 
    Gnormal=nx.Graph()
    Gred=nx.Graph()
    read_network()
    read_xy()

    for nodeinf in node_detail:
        node=nodeinf[0,0]
        Gnormal.add_node(node,pos=(nodexy[node][0],nodexy[node][1]))
        Gred.add_node(node,pos=(nodexy[node][0],nodexy[node][1]))
#         Gnormal.add_node(node,pos=(nodexy[node][1],nodexy[node][0]))
#         Gred.add_node(node,pos=(nodexy[node][1],nodexy[node][0]))
    counter=0
    red_link=[]
    normal_link=[]
    for linkinf in link_detail: 
        if counter in linklist:
            red_link.append((linkinf[0,0],linkinf[0,1]))
            Gred.add_edge(linkinf[0,0],linkinf[0,1])        
        else:
            normal_link.append((linkinf[0,0],linkinf[0,1]))
            Gnormal.add_edge(linkinf[0,0],linkinf[0,1])
        counter=counter+1
            
    pos=nx.get_node_attributes(Gnormal,'pos')
    nx.draw(Gnormal,pos,node_size=1)
    nx.draw(Gred,pos,edge_color='r',width=4,node_size=4)
    plt.savefig(title)
    #pdb.set_trace()
    #plt.close()
    #nx.draw_networkx_nodes(G,pos,node_size=50)
#     plt.draw()
#     plt.show()
    return

In [10]:
def snow_penalty(service_finished_time,snow_detail,sa_factors):
    '''
    input: 
        service_finished_time: the expected service finished time of each link
        snow_precipitation: The snow precipitation rate 
        sa_factors: The speed/capacity reduction rate caused by different level of snow accumulation
    Output:   
        Tot_Delay: delay due to the snow accumulation
    Intermediate variable: 
        snow_depth[]: snowdepth of each lane at current time interval
    '''
    average_snow_depth=[]
    j=0
    Tot_Delay=0
    for link in link_detail: #Calculate the penalty on each link
        #Calculate the reduced speed based on the snow depth
        st =service_finished_time[j]
        snow_depth_temp=0
        snow_depth=[0]*math.floor(link[0,5])
        for t in range (math.floor(num_simulation_interval)):
            penaltytemp=0
            ####Calculate the snow depth
            for snow in snow_detail:
                if snow[2] > (t+0.5)*simulation_length and snow[1] < (t+0.5)*simulation_length:
                    current_snow=snow[0]
            for i in range(math.floor(link[0,5])):
                if service_finished_time[j][i] <(t+1)*simulation_length and service_finished_time[j][i] >t*simulation_length:
                    snow_depth[i]=0
                else:
                    snow_depth[i]=snow_depth[i]+current_snow*snow_interval/60
            average_snow_depth=sum(snow_depth)/link[0,5]
            ####Finish calculating the snow depth
            
            ### Calculate the affected speed
            origin_speed=link_speed[math.floor(t*simulation_length),j]
            k=0
            while average_snow_depth>sa_factors[k][0]:
                k=k+1
            speed_reduction=sa_factors[k][1]
            capacity_reduction=sa_factors[k][2]
            reduced_speed=origin_speed*speed_reduction #account for the speed reduction
            #reduced_speed= #account for the capacity reduction use the BPR function
            penaltytemp=link_volume[math.floor(t*simulation_length),j]*(origin_speed-reduced_speed)/origin_speed*simulation_length
            Tot_Delay=Tot_Delay+penaltytemp
        j=j+1
    return Tot_Delay
#def sp_cost(start_node,target_node)
def link_snow_penalty (link,service_finished_time,last_service_finished_time):
    '''
    This subroutine update the total snow_penalty when only service time of one link is updated

    input: 
        service_finished_time: the expected service finished time of each link
        snow_precipitation: The snow precipitation rate 
        sa_factors: The speed/capacity reduction rate caused by different level of snow accumulation
    Output:   
        Tot_Delay: delay due to the snow accumulation
    Intermediate variable: 
        snow_depth[]: snowdepth of each lane at current time interval
    '''
    #Calculate the reduced speed based on the snow depth
    link_delay=0
    snow_depth_temp=0
    snow_depth=[0]*math.floor(link_detail[link,5])
    for t in range (math.floor(num_simulation_interval)):
        penaltytemp=0
        ####Calculate the snow depth
        
        for snow in snow_detail:
            if snow[2] > (t+0.5)*simulation_length and snow[1] < (t+0.5)*simulation_length:
                current_snow=snow[0]
        for i in range(math.floor(link_detail[link,5])):
            if (t+1)*simulation_length<last_service_finished_time[link][i]:
                snow_depth[i]=0
            else:
                if service_finished_time[link][i] <=(t+1)*simulation_length and service_finished_time[link][i] >=t*simulation_length:
                    snow_depth[i]=0
                else:
                    snow_depth[i]=snow_depth[i]+current_snow*snow_interval/60
        
        ####Finish calculating the snow depth

        ### Calculate the affected speed
        origin_speed=link_speed[math.floor(t*simulation_length),link]
        for depth in snow_depth:
            if depth==0:
                speed_reduction=1
                capacity_reduction=1
            else:
                k=0
                while depth>sa_factors[k][0]:
                    k=k+1
                speed_reduction=sa_factors[k][1]
                capacity_reduction=sa_factors[k][2]
            reduced_speed=origin_speed*speed_reduction #account for the speed reduction
            #reduced_speed= #account for the capacity reduction use the BPR function
            penaltytemp=link_volume[math.floor(t*simulation_length),link]*(origin_speed-reduced_speed)/origin_speed*simulation_length/link_detail[link,5]
            link_delay=link_delay+penaltytemp
    return link_delay
    

def calculate_service_finished_time ():
    '''
        Input: 
            Snowroutes: A list of nodes indicating the order of snow plow service
        Output: 
            service_finished_time: The service finish time of each lane
    '''
    return
# def path_scan(DepotLocation,LinkList):
#     '''
#     Input: 
#     DepotLocation: The node number of the depot
#     LinkList: The list of link to be plowed by the snowplow
#     Output: 
#     RoutePlan： A list of nodes indicating the working order of snow plow
#     '''
#     #Select seed node 
#     seed_node=random.choice(LinkList)
    
#     #Calculate the objective function
#     objvalue=snow_penalty()
#     #For all different priority classes construct the inital routes
#         #Set P empty set
#         #for all links
#             #Find the arc ij closet to depot/end node of class k-1 while optimize objective function
#             #If found set end=j
#         #Set P=(P,ij)
#         #for all (end,i) 
#             #find the end,i that maximizes the number of non-serviced required arc of classs k adjacent to vei
#             #set endtemp=i
#         #if endtemp ==0 then 
#             #P=P+SP(end,0)
#             #else
#             #P=P+(end,endtemp)
#             #end= endtemp
#     #Then for all non-serviced arcs ij
#         #add iji to the routes
#     #Then for all non serviced arc ij
#         #find the insertion place minimize bjective function 
#     return RoutePlan
def C_link_to_link_distance(ServiceZone):
    '''
        input: ServiceZone: sorted list of link ID, indicating the links that will be served
    '''
    link_to_link=[[0 for x in range(len(ServiceZone))] for y in range(len(ServiceZone))] 
    i=0
    link_to_link={}
    for linko in ServiceZone:
        link_to_link[linko]={}
        for linkd in ServiceZone[i+1:]:
            distance=distance_between_link(linko,linkd,link_xy)
            link_to_link[linko].update({linkd:distance})
        i=i+1
    return link_to_link



In [11]:
def timestamp(s,calculation_start_time):
    t = time.gmtime(time.clock() - calculation_start_time)
    print ('biggraph', s.ljust(24), time.strftime('%H:%M:%S', t))

In [12]:
def path_scan_heuristic(DepotLocation,ServiceZone,service_start_time,upgrade_option):
    '''
    Input: 
        DepotLocation: The node number of the depot
        LinkList: The list of link to be plowed by the snowplow
        service_start_time: The time the first snowplow leave the depot
    Output: 
        RoutePlan: A list of links indicating the working order of snow plow
    '''
    #global non_serviced_arcs,RoutePlan
    ServiceZone=sorted(ServiceZone)
    minobj=9*10**9
    mindelay=9*10**9
    non_serviced_arcs=[]
    RoutePlan=[]
    #Create a link to link distance matrix
    link_to_link=C_link_to_link_distance(ServiceZone)
        
    #Duplicate arcs with i lanes into i links
    
    for link in ServiceZone: 
        for i in range(math.floor(link_detail[link,5])):
            non_serviced_arcs.extend([link])
    
    total_num_arcs=len(non_serviced_arcs)
    num_served_arcs=0
    link_delay={}

    for link in ServiceZone:
        link_delay.update({link:link_snow_penalty(link,service_finished_time,last_service_finished_time)})
    deadhead_distance=[1000]*(len(non_serviced_arcs))
    
    while non_serviced_arcs:
        linktemp=9000000
        for link in non_serviced_arcs:
            if link !=linktemp:
                linktemp=link
                target_lane=math.floor(float(np.argmax(service_finished_time[link])))
                #Find the link that minimize the objective function
                if not RoutePlan: #if the route plan is empty, then identify the first link
                    sv_time_temp=service_finished_time[link][target_lane]
                    #Calculate the travel time from current node to the target link
                    if link_detail[link,0] != DepotLocation:
                        distance_temp=math.sqrt((link_xy[link][0]-nodexy[DepotLocation][0])**2+(link_xy[link][1]-nodexy[DepotLocation][1])**2)*1.3
                        service_finished_time[link][target_lane]=service_start_time+distance_temp/d_speed+link_detail[link,4]/s_speed/5280
                    else:
                        distance_temp=0
                        service_finished_time[link][target_lane]=service_start_time+link_detail[link,4]/s_speed/5280
                    link_delay_temp=link_delay[link]    
                    deadhead_temp=deadhead_distance[num_served_arcs]
                    link_delay[link]=link_snow_penalty(link,service_finished_time,last_service_finished_time)
                    deadhead_distance[num_served_arcs]=distance_temp
                    objtemp=sum(link_delay.values()) *VOT+sum(deadhead_distance)/d_speed*OpCost 
                    if objtemp<minobj:
                        minobj=objtemp
                        mindelay=sum(link_delay.values()) *VOT
                        LinkCandidate=link
                        can_serviced_time=service_finished_time[link][target_lane]
                        can_delay_time=link_snow_penalty(link,service_finished_time,last_service_finished_time)
                        can_target_lane=target_lane
                        can_deadhead=distance_temp
                    service_finished_time[link][target_lane]=sv_time_temp
                    link_delay[link]=link_delay_temp
                    deadhead_distance[num_served_arcs]=deadhead_temp
                    onode=DepotLocation
                elif (link_detail[link,1]!= link_detail[RoutePlan[-1],0] or link_detail[link,0]!= link_detail[RoutePlan[-1],1] )and link != RoutePlan[-1]:
                    sv_time_temp=service_finished_time[link][target_lane]
                    #Calculate the travel time from current node to the target link
                    if link_detail[link,0] != link_detail[RoutePlan[-1],1]:
                        distance_temp=link_to_link[min(RoutePlan[-1],link)][max(RoutePlan[-1],link)]*1.3
                        service_finished_time[link][target_lane]=service_start_time+distance_temp/d_speed+link_detail[link,4]/s_speed/5280
                    else:
                        distance_temp=0
                        service_finished_time[link][target_lane]=service_start_time+link_detail[link,4]/s_speed/5280
                    link_delay_temp=link_delay[link]    
                    link_delay[link]=link_snow_penalty(link,service_finished_time,last_service_finished_time)
                    
                    deadhead_temp=deadhead_distance[num_served_arcs]
                    deadhead_distance[num_served_arcs]=distance_temp
                    objtemp=sum(link_delay.values()) *VOT+sum(deadhead_distance)/d_speed*OpCost 

                    if objtemp<minobj:                    
                        minobj=objtemp
                        mindelay=sum(link_delay.values()) *VOT
                        LinkCandidate=link
                        can_serviced_time=service_finished_time[link][target_lane]
                        can_delay_time=link_snow_penalty(link,service_finished_time,last_service_finished_time)
                        can_target_lane=target_lane
                        can_deadhead=distance_temp
                        
                    service_finished_time[link][target_lane]=sv_time_temp
                    link_delay[link]=link_delay_temp
                    deadhead_distance[num_served_arcs]=deadhead_temp
                    onode=link_detail[RoutePlan[-1],1]
                  
        #Updated the route plan, service finished time. Remove selected link from non serviced arcs list
        if LinkCandidate not in non_serviced_arcs: 
            #If the link is the last one that has not been served
            if LinkCandidate==RoutePlan[-1] and link_detail[non_serviced_arcs[0],0]== link_detail[LinkCandidate,1] and link_detail[non_serviced_arcs[0],1]== link_detail[LinkCandidate,0]:
                LinkCandidate=non_serviced_arcs[0]
            else:
                print('The current link candidate is not a non serviced required arc')
                print(LinkCandidate,service_finished_time[LinkCandidate])
                pdb.set_trace()
        dnode=link_detail[LinkCandidate,0]
        deadhead_route=nx.shortest_path(Chicago_Full_Network, source=onode, target=dnode, weight='weight')
        nn=0
        #if  RoutePlan: #don't consider deadheading from depot to the first link
        for n in deadhead_route[:-1]:
            l=link_id[node_id[n],node_id[deadhead_route[nn+1]]]
#             if l==0: 
#                 pdb.set_trace()
            nn=nn+1
            if upgrade_option==1: 
                if l in non_serviced_arcs:
                    non_serviced_arcs.remove(l)
                    num_served_arcs=num_served_arcs+1
                    for i in range(3):
                        if num_served_arcs ==math.floor(total_num_arcs/4)*(i+1):
                            print('    Initial route construction',(i+1)*25,'percent completed. ',num_served_arcs,'ars are served')
            RoutePlan.extend([math.floor(l)])  #Add the deadheading to the path
        #if  RoutePlan:#don't consider deadheading from depot to the first link
#         if l==0: 
#             print(123)
#             pdb.set_trace()
        RoutePlan.extend([math.floor(LinkCandidate)])
        
        service_finished_time[LinkCandidate][can_target_lane]=can_serviced_time
        
        non_serviced_arcs.remove(LinkCandidate)
#         print(LinkCandidate,'is selected')
        service_start_time=can_serviced_time
        link_delay[LinkCandidate]=link_snow_penalty(LinkCandidate,service_finished_time,last_service_finished_time)
        deadhead_distance[num_served_arcs]=can_deadhead
        num_served_arcs=num_served_arcs+1
        for i in range(3):
            if num_served_arcs ==math.floor(total_num_arcs/4)*(i+1):
                print('    Initial route construction',(i+1)*25,'percent completed. ',num_served_arcs,'ars are served')
    print('    Initial route construction',100,'percent completed. ',num_served_arcs,'ars are served')
    return RoutePlan

In [13]:
def static_path_scan_heuristic(DepotLocation,ServiceZone,service_start_time,upgrade_option):
    '''
    Input: 
        DepotLocation: The node number of the depot
        LinkList: The list of link to be plowed by the snowplow
        service_start_time: The time the first snowplow leave the depot
    Output: 
        RoutePlan: A list of links indicating the working order of snow plow
    '''
    global static_RoutePlan
    #global non_serviced_arcs,RoutePlan
    ServiceZone=sorted(ServiceZone)
    minobj=9*10**9
    mindelay=9*10**9
    non_serviced_arcs=[]
    static_RoutePlan=[]
    #Create a link to link distance matrix
    link_to_link=C_link_to_link_distance(ServiceZone)
        
    #Duplicate arcs with i lanes into i links
    
    for link in ServiceZone: 
        for i in range(math.floor(link_detail[link,5])):
            non_serviced_arcs.extend([link])
    
    total_num_arcs=len(non_serviced_arcs)
    num_served_arcs=0
    link_delay={}

#     for link in ServiceZone:
#         link_delay.update({link:link_snow_penalty(link,service_finished_time,last_service_finished_time)})
    deadhead_distance=[1000]*(len(non_serviced_arcs))
    
    while non_serviced_arcs:
        linktemp=9000000
        for link in non_serviced_arcs:
            if link !=linktemp:
                linktemp=link
                target_lane=math.floor(float(np.argmax(service_finished_time[link])))
                #Find the link that minimize the objective function
                if not static_RoutePlan: #if the route plan is empty, then identify the first link
                    sv_time_temp=service_finished_time[link][target_lane]
                    #Calculate the travel time from current node to the target link
                    if link_detail[link,0] != DepotLocation:
                        distance_temp=math.sqrt((link_xy[link][0]-nodexy[DepotLocation][0])**2+(link_xy[link][1]-nodexy[DepotLocation][1])**2)*1.3
#                         service_finished_time[link][target_lane]=service_start_time+distance_temp/d_speed+link_detail[link,4]/s_speed/5280
                    else:
                        distance_temp=0
#                         service_finished_time[link][target_lane]=service_start_time+link_detail[link,4]/s_speed/5280
#                     link_delay_temp=link_delay[link]    
                    deadhead_temp=deadhead_distance[num_served_arcs]
#                     link_delay[link]=link_snow_penalty(link,service_finished_time,last_service_finished_time)
                    deadhead_distance[num_served_arcs]=distance_temp
                    objtemp=sum(deadhead_distance)/d_speed*OpCost 
                    if objtemp<minobj:
                        minobj=objtemp
#                         mindelay=sum(link_delay.values()) *VOT
                        LinkCandidate=link
#                         can_serviced_time=service_finished_time[link][target_lane]
#                         can_delay_time=link_snow_penalty(link,service_finished_time,last_service_finished_time)
                        can_target_lane=target_lane
                        can_deadhead=distance_temp
#                     service_finished_time[link][target_lane]=sv_time_temp
#                     link_delay[link]=link_delay_temp
                    deadhead_distance[num_served_arcs]=deadhead_temp
                    onode=DepotLocation
                elif (link_detail[link,1]!= link_detail[static_RoutePlan[-1],0] or link_detail[link,0]!= link_detail[static_RoutePlan[-1],1] )and link != static_RoutePlan[-1]:
#                     sv_time_temp=service_finished_time[link][target_lane]
                    #Calculate the travel time from current node to the target link
                    if link_detail[link,0] != link_detail[static_RoutePlan[-1],1]:
                        distance_temp=link_to_link[min(static_RoutePlan[-1],link)][max(static_RoutePlan[-1],link)]*1.3
#                         service_finished_time[link][target_lane]=service_start_time+distance_temp/d_speed+link_detail[link,4]/s_speed/5280
                    else:
                        distance_temp=0
#                         service_finished_time[link][target_lane]=service_start_time+link_detail[link,4]/s_speed/5280
#                     link_delay_temp=link_delay[link]    
#                     link_delay[link]=link_snow_penalty(link,service_finished_time,last_service_finished_time)
                    
                    deadhead_temp=deadhead_distance[num_served_arcs]
                    deadhead_distance[num_served_arcs]=distance_temp
                    objtemp=sum(deadhead_distance)/d_speed*OpCost 

                    if objtemp<minobj:                    
                        minobj=objtemp
#                         mindelay=sum(link_delay.values()) *VOT
                        LinkCandidate=link
#                         can_serviced_time=service_finished_time[link][target_lane]
#                         can_delay_time=link_snow_penalty(link,service_finished_time,last_service_finished_time)
                        can_target_lane=target_lane
                        can_deadhead=distance_temp
                        
#                     service_finished_time[link][target_lane]=sv_time_temp
#                     link_delay[link]=link_delay_temp
                    deadhead_distance[num_served_arcs]=deadhead_temp
                    onode=link_detail[static_RoutePlan[-1],1]
                  
        #Updated the route plan, service finished time. Remove selected link from non serviced arcs list
        if LinkCandidate not in non_serviced_arcs: 
            #If the link is the last one that has not been served
            if LinkCandidate==static_RoutePlan[-1] and link_detail[non_serviced_arcs[0],0]== link_detail[LinkCandidate,1] and link_detail[non_serviced_arcs[0],1]== link_detail[LinkCandidate,0]:
                LinkCandidate=non_serviced_arcs[0]
            else:
                print('The current link candidate is not a non serviced required arc')
                print(LinkCandidate,service_finished_time[LinkCandidate])
                pdb.set_trace()
        dnode=link_detail[LinkCandidate,0]
        deadhead_route=nx.shortest_path(Chicago_Full_Network, source=onode, target=dnode, weight='weight')
        nn=0
        #if  RoutePlan: #don't consider deadheading from depot to the first link
        for n in deadhead_route[:-1]:
            l=link_id[node_id[n],node_id[deadhead_route[nn+1]]]
#             if l==0: 
#                 pdb.set_trace()
            nn=nn+1
            if upgrade_option==1: 
                if l in non_serviced_arcs:
                    non_serviced_arcs.remove(l)
                    num_served_arcs=num_served_arcs+1
                    for i in range(3):
                        if num_served_arcs ==math.floor(total_num_arcs/4)*(i+1):
                            print('    Initial route construction',(i+1)*25,'percent completed. ',num_served_arcs,'ars are served')
            static_RoutePlan.extend([math.floor(l)])  #Add the deadheading to the path
        #if  RoutePlan:#don't consider deadheading from depot to the first link
#         if l==0: 
#             print(123)
#             pdb.set_trace()
        static_RoutePlan.extend([math.floor(LinkCandidate)])
        
#         service_finished_time[LinkCandidate][can_target_lane]=can_serviced_time
        
        non_serviced_arcs.remove(LinkCandidate)
#         print(LinkCandidate,'is selected')
#         service_start_time=can_serviced_time
#         link_delay[LinkCandidate]=link_snow_penalty(LinkCandidate,service_finished_time,last_service_finished_time)
        deadhead_distance[num_served_arcs]=can_deadhead
        num_served_arcs=num_served_arcs+1
        for i in range(3):
            if num_served_arcs ==math.floor(total_num_arcs/4)*(i+1):
                print('    Initial route construction',(i+1)*25,'percent completed. ',num_served_arcs,'ars are served')
    print('    Initial route construction',100,'percent completed. ',num_served_arcs,'ars are served')
    return static_RoutePlan

In [14]:
def get_start_time():
    global service_start_time
    if dynasmart_method==0: 
        snow_depth=0
        i=0
        while snow_depth<=0.2:
            snow_depth=snow_detail[i][0]/60*5+snow_depth
            i=i+1
        service_start_time=snow_detail[i][1]
    elif dynasmart_method==1:
        #read the service start time
        file=open('CurrentDynasmartTime.dat')
        for line in file: 
            service_start_time=float(line)
    

In [15]:
def calculate_snowplow_route():
    #version_number: 0 static, 1 dynamic
    service_finished_time=[]
    i=0
    for  link in link_detail:
        service_finished_time.append([])
        service_finished_time[i].extend([planning_horizon]*math.floor(link[0,5]))
        i=i+1
    i=0
    RoutePlan=[]
    calculation_start_time = time.clock()
    for (zone,depot) in zip(sub_service_clusters,depots):
        RoutePlan.append([])
        
        j=0

        for cluster in zone:
            word='Find route for zone '+str(i)+' snowplow '+str(j)
            print(word)
            RoutePlan[i].append(path_scan_heuristic(depot,cluster,service_start_time,1))
            timestamp(word,calculation_start_time)
            j=j+1
        i=i+1
    return RoutePlan
#path_scan_heuristic(depots[0],sub_service_clusters[0][3],480,1)

In [16]:
def calculate_static_snowplow_route():
    #version_number: 0 static, 1 dynamic
    service_finished_time=[]
    i=0
    for  link in link_detail:
        service_finished_time.append([])
        service_finished_time[i].extend([planning_horizon]*math.floor(link[0,5]))
        i=i+1
    i=0
    RoutePlan=[]
    calculation_start_time = time.clock()
    for (zone,depot) in zip(sub_service_clusters,depots):
        RoutePlan.append([])
        
        j=0

        for cluster in zone:
            word='Find route for zone '+str(i)+' snowplow '+str(j)
            print(word)
            RoutePlan[i].append(static_path_scan_heuristic(depot,cluster,service_start_time,1))
            timestamp(word,calculation_start_time)
            j=j+1
        i=i+1
    return RoutePlan

In [17]:
def initial_last_service_time():
    global last_service_finished_time
    last_service_finished_time=[]
    i=0
    for link in link_detail:
        last_service_finished_time.append([])
        last_service_finished_time[i].extend([planning_horizon]*math.floor(link[0,5]))
        i=i+1
    return

def update_last_service_time(service_start_time,last_service_finished_time):

    for i in range(4):
        for j in range(9):
            route=RoutePlan[i][j]
#             print(" ".join(str(x) for x in RoutePlan[i][j]),file=SPRFile)
            service_finish_time_temp=service_start_time
            for link in route: 
                if link<=num_links:
                    service_finish_time_temp=service_finish_time_temp+link_detail[link,4]/5280/0.5
                    last_service_finished_time[link][math.floor(float(np.argmax(last_service_finished_time[link])))]=service_finish_time_temp
    return           

In [19]:
# Initial settings and read inputs
print('Read the network.dat')
read_network()
# Read the traffic volume outflow.dat
# print('Read the OutFlow.dat')
read_flow()
read_speed()
print('Read the xy.dat')
read_xy()
# Parameter Setting

# The input
global s_speed,d_speed,num_simulation_interval,simulation_length,planning_horizon,cluster_method
s_speed= 0.5 #mile per minute
d_speed= 0.5 #mile per minute
Num_Zones=4
Num_Snowplows=10
num_link_classes=3
simulation_length=5  #the length of time interval that the snow accumulation updated
num_simulation_interval=1440/simulation_length
cluster_method=0 #0 if use integer programming, 1 if used the heuristic
dynasmart_method=0 #0 if use with dynasmart p, 1 if use with dynasmart x

dynasmart_simulation_length=0.1
dynasmart_num_simulation_interval=1440/dynasmart_simulation_length
num_cluster_in_zone=9
planning_horizon=1440.0
VOT=0.1 # $/minute
OpCost=0.25 # $/minute

service_start_time=480

full_network_list=[i for i in range(num_links)]
c_link_xy(full_network_list)
depots=[identify_center_depot(full_network_list)]
print('Read the snow information')
read_snow()
read_snowaccum()

#Intial Value for the service finish time of each lane 
service_finished_time=[]
i=0
for link in link_detail:
    service_finished_time.append([])
    service_finished_time[i].extend([planning_horizon]*math.floor(link[0,5]))
    i=i+1

Read the network.dat
218
Read the xy.dat
Read the snow information


In [20]:
# Divided the link into different catogories
# print('Calibrate Service Class')
# link_class_partrition()

#Determine the subzone covered by each depot
print('Determing subzone covered by depot each depot')
#sub_service_zone=net_work_partrition(subzone,len(depots),[],depots)
sub_service_zone=net_work_partrition(full_network_list,Num_Zones,depots[0],[])

##################
#identify centerilzed depots for each zone
depots=[]
for zone in sub_service_zone: 
    depots.extend([math.floor(identify_center_depot(zone))])
##################


Determing subzone covered by depot each depot
The following links are selected as seed link: 
 [4770, 1, 1014, 1659]
Solution Status: Optimal


In [21]:
###############
#This version without the consider link clas while partrition
###############
print('Determing subcluster covered by each snowplow')
sub_service_clusters=[]  # sub_service_clusters[i][j] includes all the links served within zone i by snowplow j
k=0
for subzone in sub_service_zone:
    sub_service_clusters.append([])
    if cluster_method==0: #if cluster with integer programming
    #identify the depot
        plow_zone_temp1=net_work_partrition(subzone,3,[],[])
        for temp_zone in plow_zone_temp1:
            plow_zone_temp2=net_work_partrition(temp_zone,3,[],[])
            for temp_zone2 in plow_zone_temp2:
                sub_service_clusters[k].append(temp_zone2)
        k=k+1
    elif cluster_method==1: #if cluster with the closest heuristics
        sub_service_clusters.append(net_work_partrition(subzone,num_cluster_in_zone,[],[]))
        k=k+1
    #sub_service_clusters.append([net_work_partrition_vehicle()])
    #sub_service_clusters=net_work_partrition_vehicle(sub_service_zone[0],Num_Snowplows,depot1,[])

Determing subcluster covered by each snowplow
The following links are selected as seed link: 
 [3020, 4770, 3912]
Solution Status: Optimal
The following links are selected as seed link: 
 [4770, 2843, 4251]
Solution Status: Optimal
The following links are selected as seed link: 
 [4787, 4621, 4771]
Solution Status: Optimal
The following links are selected as seed link: 
 [3660, 4772, 2844]
Solution Status: Optimal
The following links are selected as seed link: 
 [266, 1294, 73]
Solution Status: Optimal
The following links are selected as seed link: 
 [386, 1, 559]
Solution Status: Optimal
The following links are selected as seed link: 
 [1294, 698, 601]
Solution Status: Optimal
The following links are selected as seed link: 
 [4, 449, 74]
Solution Status: Optimal
The following links are selected as seed link: 
 [76, 2934, 2842]
Solution Status: Optimal
The following links are selected as seed link: 
 [1570, 27, 1521]
Solution Status: Optimal
The following links are selected as seed lin

In [22]:
global Chicago_Full_Network
Chicago_Full_Network=nx.DiGraph()
Chicago_Full_Network.add_nodes_from(math.floor(node_detail[i,0]) for i in range(num_nodes))
Chicago_Full_Network.add_edges_from([(math.floor(link_detail[i,0]),math.floor(link_detail[i,1]),{'weight':link_detail[i,4]}) for i in range(num_links)])
manual_edges=[[15293,15315],[12816,12828],[13531,13513],[13643,13642],[16115,16135],[12859,12858]]
i=1
for edge in manual_edges: 
    link_id[node_id[edge[0]],node_id[edge[1]]]=num_links+i
    i=i+1
Chicago_Full_Network.add_edges_from(manual_edges,{'weight':2480})

In [23]:
initial_last_service_time()

get_start_time()

print(dynasmart_method)
if dynasmart_method==1: #run with dynasmart x
    while service_start_time !=1440:
        while not (os.path.isfile('../DYNASMART/CurrentDynasmartTime.dat')) :
            ttt=time.clock()
        # copy the needed files 
        # CurrentDynasmartTime.dat
        # OutFlow.dat
        # OutLinkSpeedAll    
        copyfile('../DYNASMART/CurrentDynasmartTime.dat','CurrentDynasmartTime.dat')
        copyfile('../DYNASMART/OutFlowSP.dat','OutFlow.dat')
        copyfile('../DYNASMART/OutLinkSpeedAllSP.dat','OutLinkSpeedAll.dat')
        os.remove('../DYNASMART/CurrentDynasmartTime.dat')
        read_flow()
        read_speed()
        get_start_time()
        RoutePlan=calculate_snowplow_route()
        update_last_service_time(service_start_time,last_service_finished_time)

        LinkTobeServedList=[]
        SPRFile=open('SnowplowRoutes.dat','w')
        for i in range(math.floor(dynasmart_num_simulation_interval)):
            LinkTobeServedList.append([])
        print(4*num_cluster_in_zone,len(max(RoutePlan[0] ,key=len)),file=SPRFile)

        for i in range(4):
            for j in range(num_cluster_in_zone):
                route=RoutePlan[i][j]
                print(" ".join(str(x) for x in RoutePlan[i][j]),file=SPRFile)
                service_finish_time_temp=service_start_time
                for link in route: 
                    if link<=num_links:
                        service_finish_time_temp=service_finish_time_temp+link_detail[link,4]/5280/0.5
                        LinkTobeServedList[math.floor(service_finish_time_temp/simulation_length)].extend([link])

        LtbSFile=open('LinkToBeServed.dat','w')
        print(dynasmart_num_simulation_interval,len(max(LinkTobeServedList)),file=LtbSFile)
        for i in range(math.floor(dynasmart_num_simulation_interval)):
            LinkTobeServedList[i].extend([-1]*(len(max(LinkTobeServedList))-len(LinkTobeServedList[i])))
            print (" ".join(str(x) for x in LinkTobeServedList[i]),file=LtbSFile)
        LtbSFile.close() 
        SPRFile.close() 
        if os.path.isfile('../DYNASMART/LinkToBeServed.dat'):
            os.remove('../DYNASMART/LinkToBeServed.dat')  
        if os.path.isfile('../DYNASMART/SnowplowRoutes.dat'):
            os.remove('../DYNASMART/SnowplowRoutes.dat')

        copyfile('LinkToBeServed.dat','../DYNASMART/LinkToBeServed.dat')  
        copyfile('SnowplowRoutes.dat','../DYNASMART/SnowplowRoutes.dat')
        os.remove('OutLinkSpeedAll.dat')
        os.remove('OutFlow.dat')
elif dynasmart_method==0:  #run with dynasmart p            
        get_start_time()
        print(123)
        RoutePlan=calculate_snowplow_route()
        print(1234)
        update_last_service_time(service_start_time,last_service_finished_time)
        LinkTobeServedList=[]
        SPRFile=open('SnowplowRoutes.dat','w')
        for i in range(math.floor(dynasmart_num_simulation_interval)):
            LinkTobeServedList.append([])
        print(4*num_cluster_in_zone,len(max(RoutePlan[0] ,key=len)),file=SPRFile)

        for i in range(4):
            for j in range(num_cluster_in_zone):
                route=RoutePlan[i][j]
                print(" ".join(str(x) for x in RoutePlan[i][j]),file=SPRFile)
                service_finish_time_temp=service_start_time
                for link in route: 
                    if link<=num_links:
                        service_finish_time_temp=service_finish_time_temp+link_detail[link,4]/5280/0.5
                        LinkTobeServedList[math.floor(service_finish_time_temp/simulation_length)].extend([link])

        LtbSFile=open('LinkToBeServed.dat','w')
        print(dynasmart_num_simulation_interval,len(max(LinkTobeServedList)),file=LtbSFile)
        for i in range(math.floor(dynasmart_num_simulation_interval)):
            LinkTobeServedList[i].extend([-1]*(len(max(LinkTobeServedList))-len(LinkTobeServedList[i])))
            print (" ".join(str(x) for x in LinkTobeServedList[i]),file=LtbSFile)
        LtbSFile.close() 
        SPRFile.close() 

0
123
Find route for zone 0 snowplow 0
    Initial route construction 25 percent completed.  19 ars are served
    Initial route construction 50 percent completed.  38 ars are served
    Initial route construction 75 percent completed.  57 ars are served
    Initial route construction 100 percent completed.  79 ars are served
biggraph Find route for zone 0 snowplow 0 00:00:54
Find route for zone 0 snowplow 1
    Initial route construction 25 percent completed.  29 ars are served
    Initial route construction 50 percent completed.  58 ars are served
    Initial route construction 75 percent completed.  87 ars are served
    Initial route construction 100 percent completed.  117 ars are served
biggraph Find route for zone 0 snowplow 1 00:03:33
Find route for zone 0 snowplow 2
    Initial route construction 25 percent completed.  47 ars are served
    Initial route construction 50 percent completed.  94 ars are served
    Initial route construction 75 percent completed.  141 ars are serv

In [33]:
dynasmart_num_simulation_interval

288.0

In [37]:


service_start_time=443
dynasmart_num_simulation_interval=14400
simulation_length=0.1
update_last_service_time(service_start_time,last_service_finished_time)
LinkTobeServedList=[]
SPRFile=open('SnowplowRoutes.dat','w')
for i in range(math.floor(dynasmart_num_simulation_interval)):
    LinkTobeServedList.append([])
print(4*num_cluster_in_zone,len(max(RoutePlan[0] ,key=len)),file=SPRFile)

for i in range(4):
    for j in range(num_cluster_in_zone):
        route=RoutePlan[i][j]
        print(" ".join(str(x) for x in RoutePlan[i][j]),file=SPRFile)
        service_finish_time_temp=service_start_time
        for link in route: 
            if link<=num_links:
                service_finish_time_temp=service_finish_time_temp+link_detail[link,4]/5280/0.5
                
                LinkTobeServedList[math.floor(service_finish_time_temp/simulation_length)].extend([link])

LtbSFile=open('LinkToBeServed.dat','w')
print(dynasmart_num_simulation_interval,len(max(LinkTobeServedList)),file=LtbSFile)
for i in range(math.floor(dynasmart_num_simulation_interval)):
    LinkTobeServedList[i].extend([-1]*(len(max(LinkTobeServedList))-len(LinkTobeServedList[i])))
    print (" ".join(str(x) for x in LinkTobeServedList[i]),file=LtbSFile)
LtbSFile.close() 
SPRFile.close() 

In [36]:
service_start_time

4430

In [24]:
initial_last_service_time()

get_start_time()

print(dynasmart_method)
if dynasmart_method==1: #run with dynasmart x
    while service_start_time !=1440:
        while not (os.path.isfile('../DYNASMART/CurrentDynasmartTime.dat')) :
            ttt=time.clock()
        # copy the needed files 
        # CurrentDynasmartTime.dat
        # OutFlow.dat
        # OutLinkSpeedAll    
        copyfile('../DYNASMART/CurrentDynasmartTime.dat','CurrentDynasmartTime.dat')
        copyfile('../DYNASMART/OutFlowSP.dat','OutFlow.dat')
        copyfile('../DYNASMART/OutLinkSpeedAllSP.dat','OutLinkSpeedAll.dat')
        os.remove('../DYNASMART/CurrentDynasmartTime.dat')
        read_flow()
        read_speed()
        get_start_time()
        RoutePlan=calculate_snowplow_route()
        update_last_service_time(service_start_time,last_service_finished_time)

        LinkTobeServedList=[]
        SPRFile=open('SnowplowRoutes.dat','w')
        for i in range(math.floor(dynasmart_num_simulation_interval)):
            LinkTobeServedList.append([])
        print(4*num_cluster_in_zone,len(max(RoutePlan[0] ,key=len)),file=SPRFile)

        for i in range(4):
            for j in range(num_cluster_in_zone):
                route=RoutePlan[i][j]
                print(" ".join(str(x) for x in RoutePlan[i][j]),file=SPRFile)
                service_finish_time_temp=service_start_time
                for link in route: 
                    if link<=num_links:
                        service_finish_time_temp=service_finish_time_temp+link_detail[link,4]/5280/0.5
                        LinkTobeServedList[math.floor(service_finish_time_temp/simulation_length)].extend([link])

        LtbSFile=open('LinkToBeServed.dat','w')
        print(dynasmart_num_simulation_interval,len(max(LinkTobeServedList)),file=LtbSFile)
        for i in range(math.floor(dynasmart_num_simulation_interval)):
            LinkTobeServedList[i].extend([-1]*(len(max(LinkTobeServedList))-len(LinkTobeServedList[i])))
            print (" ".join(str(x) for x in LinkTobeServedList[i]),file=LtbSFile)
        LtbSFile.close() 
        SPRFile.close() 
        if os.path.isfile('../DYNASMART/LinkToBeServed.dat'):
            os.remove('../DYNASMART/LinkToBeServed.dat')  
        if os.path.isfile('../DYNASMART/SnowplowRoutes.dat'):
            os.remove('../DYNASMART/SnowplowRoutes.dat')

        copyfile('LinkToBeServed.dat','../DYNASMART/LinkToBeServed.dat')  
        copyfile('SnowplowRoutes.dat','../DYNASMART/SnowplowRoutes.dat')
        os.remove('OutLinkSpeedAll.dat')
        os.remove('OutFlow.dat')
elif dynasmart_method==0:  #run with dynasmart p            
        get_start_time()
        print(123)
        static_RoutePlan=calculate_static_snowplow_route()
        print(1234)
        update_last_service_time(service_start_time,last_service_finished_time)
        LinkTobeServedList=[]
        SPRFile=open('staticSnowplowRoutes.dat','w')
        for i in range(math.floor(dynasmart_num_simulation_interval)):
            LinkTobeServedList.append([])
        print(4*num_cluster_in_zone,len(max(static_RoutePlan[0] ,key=len)),file=SPRFile)

        for i in range(4):
            for j in range(num_cluster_in_zone):
                route=static_RoutePlan[i][j]
                print(" ".join(str(x) for x in static_RoutePlan[i][j]),file=SPRFile)
                service_finish_time_temp=service_start_time
                for link in route: 
                    if link<=num_links:
                        service_finish_time_temp=service_finish_time_temp+link_detail[link,4]/5280/0.5
                        LinkTobeServedList[math.floor(service_finish_time_temp/simulation_length)].extend([link])

        LtbSFile=open('staticLinkToBeServed.dat','w')
        print(dynasmart_num_simulation_interval,len(max(LinkTobeServedList)),file=LtbSFile)
        for i in range(math.floor(dynasmart_num_simulation_interval)):
            LinkTobeServedList[i].extend([-1]*(len(max(LinkTobeServedList))-len(LinkTobeServedList[i])))
            print (" ".join(str(x) for x in LinkTobeServedList[i]),file=LtbSFile)
        LtbSFile.close() 
        SPRFile.close() 

0
123
Find route for zone 0 snowplow 0
    Initial route construction 25 percent completed.  19 ars are served
    Initial route construction 50 percent completed.  38 ars are served
    Initial route construction 75 percent completed.  57 ars are served
    Initial route construction 100 percent completed.  79 ars are served
biggraph Find route for zone 0 snowplow 0 00:00:00
Find route for zone 0 snowplow 1
    Initial route construction 25 percent completed.  29 ars are served
    Initial route construction 50 percent completed.  58 ars are served
    Initial route construction 75 percent completed.  87 ars are served
    Initial route construction 100 percent completed.  117 ars are served
biggraph Find route for zone 0 snowplow 1 00:00:00
Find route for zone 0 snowplow 2
    Initial route construction 25 percent completed.  47 ars are served
    Initial route construction 50 percent completed.  94 ars are served
    Initial route construction 75 percent completed.  141 ars are serv

In [25]:
dynasmart_method

0

In [ ]:
simulation_length=0.1
dynasmart_num_simulation_interval=int(1440.0/simulation_length)
num_links=4805
LinkTobeServedList=[]
for i in range(math.floor(dynasmart_num_simulation_interval)):
    LinkTobeServedList.append([])
count=0
for zone in static_RoutePlan:
    for route in zone:
        service_finish_time_temp=service_start_time
        for link in route: 
            
            if link<=num_links:
                service_finish_time_temp=service_finish_time_temp+link_detail[link,4]/5280/0.5
                if service_finish_time_temp>service_start_time+180:
                    service_finish_time_temp=service_start_time
                    count=count+1
                LinkTobeServedList[math.floor(service_finish_time_temp/simulation_length)].extend([link])
LtbSFile=open('D:\\alex\\AMS Project\\SP routing\\Full network - oc4\\staticLinkToBeServed.dat','w')
print(LtbSFile)
print(dynasmart_num_simulation_interval,len(max(LinkTobeServedList)),file=LtbSFile)

for i in range(math.floor(dynasmart_num_simulation_interval)):
    LinkTobeServedList[i].extend([-1]*(max(len(p) for p in LinkTobeServedList)-len(LinkTobeServedList[i])))
    print (" ".join(str(x) for x in LinkTobeServedList[i]),file=LtbSFile)

LtbSFile.close() 

In [ ]:
os.getcwd()

In [ ]:
update_last_service_time(service_start_time,last_service_finished_time)
LinkTobeServedList=[]
SPRFile=open('SnowplowRoutes.dat','w')
for i in range(math.floor(dynasmart_num_simulation_interval)):
    LinkTobeServedList.append([])
print(4*num_cluster_in_zone,len(max(RoutePlan[0] ,key=len)),file=SPRFile)

for i in range(4):
    for j in range(num_cluster_in_zone):
        route=RoutePlan[i][j]
        print(" ".join(str(x) for x in RoutePlan[i][j]),file=SPRFile)
        service_finish_time_temp=service_start_time
        for link in route: 
            if link<=num_links:
                service_finish_time_temp=service_finish_time_temp+link_detail[link,4]/5280/0.5
                LinkTobeServedList[math.floor(service_finish_time_temp/simulation_length)].extend([link])

LtbSFile=open('LinkToBeServed.dat','w')
print(dynasmart_num_simulation_interval,len(max(LinkTobeServedList)),file=LtbSFile)
for i in range(math.floor(dynasmart_num_simulation_interval)):
    LinkTobeServedList[i].extend([-1]*(len(max(LinkTobeServedList))-len(LinkTobeServedList[i])))
    print (" ".join(str(x) for x in LinkTobeServedList[i]),file=LtbSFile)
LtbSFile.close() 
SPRFile.close() 
os.remove('OutLinkSpeedAll.dat')
os.remove('OutFlow.dat')

In [ ]:
LtbSFile=open('LinkToBeServedtemp2.dat','w')
for i in range (288):
    if i <96 or i>119:
        print(" ".join(str(x) for x in [-1]*200),file=LtbSFile)
    else:
        print(" ".join(str(x) for x in [(i-96)*200+j+1 for j in range(200)]),file=LtbSFile)
LtbSFile.close()

In [ ]:
# def multi_commodity_model(cluster):
#     '''
#         input:
#             clusters: link ID of links will be covered by the depot
#             link: link length of all links within the clusters
#             s_speed: service speed
#             d_speed: deadhead speed
#         intermediat output: 
#             x: 0 or 1 integer indicate whether an arc is serviced or not
#             y: an integer indicates whether the edge is traversed by associated vehicle
#         output: 
#             route plan
#     '''
#     #Reconstruct the network
#     #Duplicate multiple lane 
#     new_cluster=[]
#     ijindicator=[]
#     for link in cluster: 
#         for lane in range(0,int(link_detail[link,5])):
#             new_cluster.append(link)
#             if link_detail[link,0]>link_detail[link,2]:
#                 ijindicator.append(1)
#             else:
#                 ijindicator.append(0)
#     #Add the depot and fictious class (needs to be done)
    
#     #Create the index for the vehicle class
#     cluster_link_index=[i for i in range(len(new_cluster))]
#     cluster_class_index=[i for i in range(3)]
    
#     #decision variables
#     service_links=pulp.LpVariable.dicts("service_link",(cluster_link_index,cluster_class_index),0,1,pulp.LpInteger)
#     traversed_links=pulp.LpVariable.dicts("traversed_link",(cluster_link_index,cluster_class_index),0,10,pulp.LpInteger)
    
#     prob=pulp.LpProblem('Snowplowrouting Problem',pulp.LpMinimize)
    
#     #input variables
#     service_time=[link_detail[i,4]/1024/s_speed for i in new_cluster]
#     deadhead_time=[link_detail[i,4]/1024/d_speed for i in new_cluster]
#     #objective function
#     prob += pulp.lpSum(service_links[l][c]*float(service_time[l])+traversed_links[l][c]*float(deadhead_time[l]) for l in cluster_link_index for c in cluster_class_index)
    
#     #constraints 2.2
#     for p in cluster_class_index:
#         if p==0:
#             prob += pulp.lpSum(service_links[l][0]*float(service_time[l])+traversed_links[l][0]*float(deadhead_time[l]) for l in cluster_link_index)>=0.0
#         else:
#             prob += pulp.lpSum(service_links[l][p]*float(service_time[l])+traversed_links[l][p]*float(deadhead_time[l]) for l in cluster_link_index)-pulp.lpSum(service_links[l][p-1]*float(service_time[l])+traversed_links[l][p-1]*float(deadhead_time[l]) for l in cluster_link_index)>=0.01
#     #Completion time 2.3
#     for l in cluster_link_index:
#         print(pulp.lpSum([service_links[l][z] for z in cluster_class_index]))
#         prob += pulp.lpSum([service_links[l][z] for z in cluster_class_index]) == 1  #2.5
#     #2.7
#     prob += pulp.lpSum(service_links[l][z]-2*service_links[l][z]*ijindicator[l] for l in cluster_link_index for z in cluster_class_index)==0
#     prob += pulp.lpSum(service_links)
#     print(service_links[l][z]*ijindicator[l] for l in cluster_link_index) #for z in cluster_class_index)
#     #2.8
    
    
    
#     prob.writeLP("Multi Commodity Routing.lp")
#     # The problem is solved using PuLP's choice of Solver
#     prob.solve()
#     print("Solution Status:", pulp.LpStatus[prob.status])
    
# #     network_link_cluster_list=[[] for i in range(num_cluster)]
# #     for l in cluster_link_index:
# #         for z in cluster_zone_index:
# #             if pulp.value(link_cluster[l][z])==1:
# #                 network_link_cluster_list[z].append(network_link_list[l])
    
#     return
    
    
    

In [ ]:
# def net_work_partrition_vehicle(network_link_list,num_cluster,center,seed_links):
#     '''
#     input: 
#         network_link_list: a list of link id that needs to be partitioned 
#         num_cluster: the network would be partitioned into num_cluster sets
#         center: optional input, the location of the depot
#         seed_links: optional input, the seed link for each cluster
#     output
#         clusters: a list of link contained in each cluster
#     '''
#     #global d
#     if not center: 
#         centertemp=random.choice(network_link_list)
#         center=link_detail[centertemp,0]
#     if not seed_links: # in this case, we do not have a seedlink,but have a center depot
#         for i in range(num_cluster):
#             maxdistance=0
#             linkcandidate=0
#             #Go over all the candidate links to find the one which locates furtherest to the depot and other seedlinks
#             for link in network_link_list:
#                 #calculate distance form link candidate to the depot
#                 distance_to_depot=math.sqrt((nodexy[link_detail[link,0]][0]-nodexy[center][0])**2
#                                             +(nodexy[link_detail[link,0]][1]-nodexy[center][1])**2)
#                 total_distance=distance_to_depot
#                 #calculate distance form link candidate to other seed links
#                 for seed_link in seed_links: 
#                     total_distance=total_distance*distance_between_link(seed_link,link,link_xy)
#                 if total_distance>maxdistance and link not in seed_links:
#                     maxdistance=total_distance
#                     linkcandidate=link
#             seed_links.append(linkcandidate)
#     print('The following links are selected as seed link: \n',seed_links)
    
#     #After the seed_links are selected, solve the integer problem to identify the link clusters
#     cluster_link_index=[i for i in range(len(network_link_list))]
#     cluster_zone_index=[i for i in range(num_cluster)]
#     link_length=[link_detail[i,4]*link_detail[i,5]/5280 for i in network_link_list]
    
#     L=float(sum(link_length)/num_cluster*0.85)
#     U=float(sum(link_length)/num_cluster*1.15)

#     prob=pulp.LpProblem('Link Cluster Problem vehicle',pulp.LpMinimize)
#     link_cluster=pulp.LpVariable.dicts("Cluster vehicle",(cluster_link_index,cluster_zone_index),0,1,pulp.LpInteger)
#     d=np.zeros(shape=(len(network_link_list),num_cluster))
#     for l in cluster_link_index:
#         for z in cluster_zone_index:
#             #print(l,z,network_link_list[l],seed_links[z])
#             d[l,z]=distance_between_link(network_link_list[l],seed_links[z],link_xy)
#     print('Write Constraints')
#     prob += pulp.lpSum([d[l,z]*link_cluster[l][z]] for l in cluster_link_index for z in cluster_zone_index), "Total distance from link to assigned seed link vehicle"

#     for l in cluster_link_index:
#         prob += pulp.lpSum([link_cluster[l][z] for z in cluster_zone_index]) == 1
#     print(2)
#     for z in cluster_zone_index:
#         prob += pulp.lpSum([float(link_length[l])*link_cluster[l][z]] for l in cluster_link_index )>=L
#         prob += pulp.lpSum([float(link_length[l])*link_cluster[l][z]] for l in cluster_link_index)<=U
#     print(3)
#     prob.writeLP("Link Cluster Problem vehicle.lp")
#     print(4)
#     # The problem is solved using PuLP's choice of Solver
    
#     prob.solve(pulp.GLPK())
#     print("Solution Status:", pulp.LpStatus[prob.status])
#     network_link_cluster_list=[[] for i in range(num_cluster)]
#     for l in cluster_link_index:
#         for z in cluster_zone_index:
#             if pulp.value(link_cluster[l][z])==1:
#                 network_link_cluster_list[z].append(network_link_list[l])
    
#     return network_link_cluster_list




In [ ]:
##################
#This version consider class priority while partrition
##################
# print('Determing subcluster covered by each snowplow')
# sub_service_clusters=[]
# k=0
# for subzone in sub_service_zone:
#     sub_service_clusters.append([])
#     #identify the depot
#     depot=identify_center_depot(subzone)
#     print("Depot for zone",k,'is',depot)
#     seed_links=[]
#     class_link=[[] for i in range(num_link_classes)]
#     for i in range(Num_Snowplows):
#         maxdistance=0
#         linkcandidate=0
#         #Go over all the candidate links to find the one which locates furtherest to the depot and other seedlinks
#         for link in subzone:
#             #calculate distance form link candidate to the depot
#             distance_to_depot=math.sqrt((nodexy[link_detail[link,0]][0]-nodexy[depot][0])**2
#                                         +(nodexy[link_detail[link,0]][1]-nodexy[depot][1])**2)
#             total_distance=distance_to_depot
#             #calculate distance form link candidate to other seed links
#             for seed_link in seed_links: 
#                 total_distance=total_distance*distance_between_link(seed_link,link,link_xy)
#             if total_distance>maxdistance and link not in seed_links:
#                 maxdistance=total_distance
#                 linkcandidate=link
#             class_link[link_class[link]-1].extend([link])
#         seed_links.append(linkcandidate)
#         sub_service_clusters[k].append([])
#     for i in range(num_link_classes):
#         plow_zone_temp1=net_work_partrition(class_link[i],Num_Snowplows/2,depot,seed_links)
#         for j in range(Num_Snowplows/2):
            
#             for m in ran
#             sub_service_clusters[k][i].extend([plow_zone_temp[j]])
#     k=k+1
#     #sub_service_clusters.append([net_work_partrition_vehicle()])
#     #sub_service_clusters=net_work_partrition_vehicle(sub_service_zone[0],Num_Snowplows,depot1,[])

In [ ]:
# def network_partrition_nearest_heuristic(network_link_list,num_cluster,center,seed_links):
#     '''
#     input: 
#         network_link_list: a list of link id that needs to be partitioned 
#         num_cluster: the network would be partitioned into num_cluster sets
#         center: optional input, the location of the depot
#         seed_links: optional input, the seed link for each cluster
#     output
#         clusters: a list of link contained in each cluster
#     '''
# #     global d
#     if not center: 
#         centertemp=random.choice(network_link_list)
#         center=link_detail[centertemp,0]
#     if not seed_links: # in this case, we do not have a seedlink,but have a center depot
#         for i in range(num_cluster):
#             maxdistance=0
#             linkcandidate=0
#             #Go over all the candidate links to find the one which locates furtherest to the depot and other seedlinks
#             for link in network_link_list:
#                 #calculate distance form link candidate to the depot
# #                 Distancecalculated based on the middle points of the link
# #                 distance_to_depot=math.sqrt(((nodexy[link_detail[link,0]][0]+nodexy[link_detail[link,1]][0])/2-nodexy[center][0])**2+
# #                                             ((nodexy[link_detail[link,0]][1]+nodexy[link_detail[link,1]][1])/2-nodexy[center][1])**2) 
#                 #Distance calculated based on the up stream node
#                 distance_to_depot=math.sqrt((nodexy[link_detail[link,0]][0]-nodexy[center][0])**2+
#                                              (nodexy[link_detail[link,0]][1]-nodexy[center][1])**2) 
#                 total_distance=distance_to_depot
#                 #calculate distance form link candidate to other seed links
#                 for seed_link in seed_links: 
#                     total_distance=total_distance*distance_between_link(seed_link,link,link_xy)
#                 if total_distance>maxdistance and link not in seed_links:
#                     maxdistance=total_distance
#                     linkcandidate=link
#             seed_links.append(linkcandidate)
#     print('The following links are selected as seed link: \n',seed_links)
    
#     #After the seed_links are selected, solve the integer problem to identify the link clusters
#     cluster_link_index=[i for i in range(len(network_link_list))]
#     cluster_zone_index=[i for i in range(num_cluster)]
#     link_length=[link_detail[i,4] for i in network_link_list]
    
# #     L=float(sum(link_length)/num_cluster*0.8)
# #     U=float(sum(link_length)/num_cluster*1.2)

# #     prob=pulp.LpProblem('Link Cluster Problem',pulp.LpMinimize)
# #     link_cluster=pulp.LpVariable.dicts("Cluster",(cluster_link_index,cluster_zone_index),0,1,pulp.LpInteger)
#     print('Prepare distance matrix between links and seed links')
#     d=np.zeros(shape=(len(network_link_list),num_cluster))
#     for l in range(len(network_link_list)):
#         for z in range(num_cluster):
#             #print(l,z,network_link_list[l],seed_links[z])
#             d[l,z]=distance_between_link(network_link_list[l],seed_links[z],link_xy)

#     network_link_cluster_list=[[] for i in range(num_cluster)]
#     for l in cluster_link_index:
#         s_distance=999999
#         c_seed=99
#         for z in cluster_zone_index:
#             if d[l,z]<s_distance:
#                 s_distance=d[l,z]
#                 c_seed=z
#         network_link_cluster_list[c_seed].append(network_link_list[l])
#     return network_link_cluster_list
